## 6-3. LSA를 이용한 차원 축소와 의미 파악
'**LSA(Latent Semantic Analysis)**'는 우리말로 '잠재 의미 분석'이다. 문서들에 잠재된 의미를 분석하고 동시에 단어들에 잠재된 의미도 분석할 수 있다. 여기서 의미는 문서와 단어를 연결하는 매개체가 되는데, 바로 축소된 차원이 그 역할을 한다. LSA는 SVD(Singular Value Decomposition), 더 정확하게는 절단된 SVD(Truncated SVD)를 이용해 구현되는데, 보다 정확한 이해를 위해서는 SVD를 먼저 이해할 필요가 있다.

SVD, 즉 특잇값 분해는 m*n의 크기를 가진 주어진 행렬을 아래와 같이 세 개의 행렬의 곱으로 분해하는 것을 말한다.

$X=U \Sigma Vᵗ$

이때 U, Vᵗ은 각각 m\*m과 n\*n의 크기를 갖는 직교행렬이어야 하고, Σ는 m*n의 크기를 갖는 대각행렬이어야 한다. 위 식과 조건에 대해 각각의 의미를 설명하는 것은 책의 범위를 넘어서므로 더 이상의 설명은 생략하지만, 특잇값 분해는 앞에서 설명한 PCA와 같이 결과적으로 공분산행렬의 고유값을 만들어내고 이를 통해 차원축소가 가능하게 해준다.

여기서 왜 차원축소를 위해 세 개의 행렬의 곱으로 분해하는지가 중요하다. 일단 위 식대로 한다면 분해된 세 개의 행렬을 다시 곱해서 원래의 데이터를 복원할 수 있다. 위 행렬에서 차원 축소의 키가 되는 행렬은 Σ이다. 여기에는 주성분 분석 결과와 동일한 정보가 들어간다. 즉 가장 정보량이 많은 축이 위에 있고 아래로 점차 줄어드는 형태로 새로운 축에 대한 정보가 들어간다고 보면 된다.

LSA에서의 차원 축소는 SVD의 변형인 절단된 SVD를 통해 이루어진다. 절단된 SVD는 차원 축소를 위해 Σ의 대각 원소인 특잇값에서 상위 k개만 골라낸다. 그 결과, U에서 k개의 열 그리고, Vᵗ에서도 k개의 행만 남게 된다. SVD에서는 세 행렬의 곱으로 원래의 X를 복원할 수 있지만 절단된 SVD에서는 완전한 복원이 불가능하고 대신 최대한 유사한 값이 되도록 할 수 있다. 즉, 정보의 손실이 발생하게 된다. 

여기까지의 설명은 선형대수에 대한 지식 없이는 사실상 완전한 이해가 불가능하다. 활용을 위해 중요한 것은 이제부터라고 할 수 있다. LSA를 사용하는 첫째 목적은 차원의 축소다. 즉 X가 각 문서에 대한 카운트 벡터 혹은 tfidf 벡터로 이루어진 행렬이라고 할 때, 각 문서는 n개의 단어 빈도를 통해 표현된다. 우리가 원하는 것은 이것을 k개의 차원으로 축소하는 것이다.

사이킷런은 [TruncatedSVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) 클래스를 이용해 문서에 대한 LSA를 지원한다. PCA 클래스에서 희소행렬을 직접 연산하지 못한 것과 달리, TruncatedSVD는 바로 연산이 가능해 toarray() 메서드를 쓸 필요가 없다. TruncatedSVD의 transform()이나 fit_transform() 메서드를 호출하면 축소된 차원의 문서 행렬을 반환해 준다. 지금까지 본 바와 같이 fit_transform()은 학습 데이터에 대해, transform()은 테스트 데이터에 대해 사용된다. TruncatedSVD는 두 메서드에 대해 위와 같이 축소된 문서 행렬을 반환한다.

위에서 축소된 차원은 잠재된 의미를 나타내고, 문서와 단어를 연결한다고 설명했다. 즉 문서 벡터가 m개의 차원일 때에는 사용된 단어의 빈도에 기반해 문서를 표현하지만, k개의 차원으로 축소한 후에는 k개의 잠재된 의미의 비중으로 각 문서를 표현한다. 다시 말해서 k개의 축소된 차원은 각각 잠재된 의미를 표현하고 있으며, 이를 단어에 대해서 적용하면 각 단어도 k개의 잠재된 의미의 비중으로 표현돼 있다고 해석할 수 있다. 단어들에 대해 잠재된 의미의 비중으로 표현된 값들을 보고 싶다면, np.diag(svd.singularvalues).dot(svd.components_)로 구할 수 있다. 여기에 대한 계산과 활용은 조금 뒤에서 실습하기로 한다.

### LSA를 이용한 차원 축소와 성능
먼저 사이킷런의 TruncatedSVD를 이용해 차원 축소를 하고, 주성분 분석의 결과와 비교해 본다. 사이킷런에서 TruncatedSVD는 사용에 있어 PCA와 중요한 차이점이 있다. TruncatedSVD는 희소 벡터에 대한 연산을 바로 지원한다. PCA는 X_train_tfidf.toarray()로 희소행렬 형식을 일반적은 numpy 행렬로 변환해야 했지만, TruncatedSVD에서는 TFIDF 희소행렬을 바로 사용할 수 있다.

차원축소를 위해서 PCA와 마찬가지로 TruncatedSVD 객체를 선언할 때 n_componets로 차원의 크기를 정한다. 주성분 분석과 동일하게 먼저 2,000개로 축소한 후, 축소된 데이터로 분류 모형을 학습한다.

In [1]:
% store -r X_train
% store -r X_test
% store -r y_train
% store -r y_test
% store -r X_train_tfidf
% store -r X_test_tfidf
% store -r newsgroups_train
% store -r newsgroups_test

In [2]:
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

svd = TruncatedSVD(n_components=2000, random_state=7) # 압축할 component의 수 지정
X_train_lsa = svd.fit_transform(X_train_tfidf)
X_test_lsa = svd.transform(X_test_tfidf)
print(f'LSA Converted X shape: {X_train_lsa.shape}')
print(f'Sum of explained variance ratio: {svd.explained_variance_ratio_.sum():.3f}')

LR_clf = LogisticRegression()
LR_clf.fit(X_train_lsa, y_train)
print(f'# Train set score: {LR_clf.score(X_train_lsa, y_train):.3f}')
print(f'# Test set score: {LR_clf.score(X_test_lsa, y_test):.3f}')

LSA Converted X shape: (2034, 2000)
Sum of explained variance ratio: 1.000
# Train set score: 0.962
# Test set score: 0.761


주성분 분석 결과와 동일하게 설명된 분산의 비율이 1.0으로 나오고, 테스트 집합에 대한 정확도도 동일하다. 사이킷런에서 자연어 처리를 할 때에는 PCA보다 TruncatedSVD를 사용하는 것이 더 일반적이다.

이제 차원의 수를 100개로 줄여본다. TruncatedSVD는 PCA와 거의 같지만 알고리즘에 미세한 차이가 있어 차원 축소 결과가 완벽하게 일치하지는 않는다. 아래 결과는 PCA보다 나은 결과를 보여주지만 random_state를 변경하면 다른 결과를 볼 수 있다.

In [3]:
svd = TruncatedSVD(n_components=100, random_state=1) # 압축할 component의 수 지정
X_train_lsa = svd.fit_transform(X_train_tfidf)
X_test_lsa = svd.transform(X_test_tfidf)
print(f'LSA Converted X shape: {X_train_lsa.shape}')
print(f'Sum of explained variance ratio: {svd.explained_variance_ratio_.sum():.3f}')

LR_clf.fit(X_train_lsa, y_train)
print(f'# Train set score: {LR_clf.score(X_train_lsa, y_train):.3f}')
print(f'# Test set score: {LR_clf.score(X_test_lsa, y_test):.3f}')

LSA Converted X shape: (2034, 100)
Sum of explained variance ratio: 0.209
# Train set score: 0.810
# Test set score: 0.745


### LSA를 이용한 의미 기반의 문서 간 유사도 계산
SVD를 자연어에 적용했을 때 LSA(잠재의미분석)라고 불리우는 이유는 축소된 차원이 잠재된 의미를 보여준다고 해석할 수 있기 때문이다. 문서에 대해 카운트 기반의 표현을 이용해 벡터로 변환하면 4장에서 살펴본 것과 같이 문서 간의 거리 혹은 유사도를 계산하는 것이 가능하다. 그 문서에 사용된 단어들의 빈도를 이용해서 유사도를 계산하는 것인데, LSA로 축소된 문서의 벡터는 단어가 아니라 잠재된 의미의 비중으로 문서를 표현하므로 이를 이용해 유사도를 계산하는 것도 가능하다.

위에서 100개의 차원으로 축소된 결과를 이용해 각 문서 간의 거리를 계산해본다. 편의를 위해 첫째 문서에 대해 전체 문서와의 유사도를 계산한다. 당연히 자기 자신에 대해서는 유사도가 1이 나올 것이다. 아래와 같이 계산된 유사도를 이용해 유사도가 높은 순으로 20개 문서의 인덱스와 카테고리를 출력한다.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

print(f'# 사용된 전체 카테고리: {newsgroups_train.target_names}')
print(f'# 첫 문서의 카테고리: {y_train[0]}')

# 변환된 count vector와 기존 값들과의 similarity 계산
sim_result = cosine_similarity([X_train_lsa[0]], X_train_lsa)

print(f'# Top 20 유사도(LSA):\n {sorted(sim_result[0].round(2), reverse=True)[:20]}')
sim_index = (-sim_result[0]).argsort()[:20]
print(f'# Top 20 유사 뉴스의 인덱스(LSA):\n {sim_index}')
sim_labels = [y_train[i] for i in sim_index]
print(f'# Top 20 유사 뉴스의 카테고리(LSA):\n {sim_labels}')

# 사용된 전체 카테고리: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
# 첫 문서의 카테고리: 1
# Top 20 유사도(LSA):
 [1.0, 0.74, 0.74, 0.72, 0.7, 0.7, 0.69, 0.67, 0.66, 0.65, 0.65, 0.65, 0.63, 0.62, 0.62, 0.62, 0.57, 0.57, 0.55, 0.54]
# Top 20 유사 뉴스의 인덱스(LSA):
 [   0 1957 1674  501 1995 1490  790 1902 1575 1209 1728  892 1892  998
 1038 1826 1290 1089  867  151]
# Top 20 유사 뉴스의 카테고리(LSA):
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


위 결과를 보면 첫째 문서의 카테고리가 1('comp.graphics')인 것을 볼 수 있다. 자신을 포함해 유사도가 높은 상위 20개 문서의 카테고리가 모두 1이므로, 유사도가 잘 작동한다고 판단할 수 있다.

그렇다면 TFIDF 벡터를 이용한 유사도는 어떨까? 아래와 같이 대상이 되는 행렬을 X_train_lsa에서 X_train_tfidf로 바꿔서 계산해보자.

In [5]:
sim_result = cosine_similarity(X_train_tfidf[0], X_train_tfidf)

print(f'# Top 20 유사도(TFIDF):\n {sorted(sim_result[0].round(2), reverse=True)[:20]}')
sim_index = (-sim_result[0]).argsort()[:20]
print(f'# Top 20 유사 뉴스의 인덱스(TFIDF):\n {sim_index}')
sim_labels = [y_train[i] for i in sim_index]
print(f'# Top 20 유사 뉴스의 카테고리(TFIDF):\n {sim_labels}')

# Top 20 유사도(TFIDF):
 [1.0, 0.3, 0.22, 0.21, 0.19, 0.19, 0.19, 0.17, 0.16, 0.16, 0.16, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.14]
# Top 20 유사 뉴스의 인덱스(TFIDF):
 [   0 1575 1892 1490  501 1290 1013  998 1636 1705 1995 1957 1664  651
 1038  429 1089 1209 1728 1803]
# Top 20 유사 뉴스의 카테고리(TFIDF):
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


LSA 결과와 원래 TFIDF에 대한 유사도 계산 결과를 비교해보면, 순위에 차이가 있으나 결과적으로 상위 20개 모두 동일한 카테고리의 문서를 가져온다. 다만 유사도 값에서 차이가 큰데, TFIDF는 자신을 제외하면 0.53부터 시작하는 반면, LSA는 0.74부터 시작하고 20개 중 가장 낮은 값도 0.54다.

즉 TFIDF는 유사도가 좁은 범위에 분포하게 되는데, 이는 차원이 상대적으로 훨씬 커서 LSA에 비해 각 문서 간 거리가 멀기 때문으로 차원의 저주가 적용됐다고 할 수 있다. 클러스터링과 같은 작업을 통해 분류 또는 시각화한다면 차원이 적은 쪽이 훨씬 유리하다. 상대적으로 문서 간의 거리가 가깝고 좁게 분포해 있어서다. 이러한 시각화는 조금 뒤에서 구체적으로 살펴본다.

### 잠재된 토픽의 분석
문서들이 잠재된 의미의 비중으로 표현되고 다시 그 잠재된 의미가 단어들과 연관되어 있다면, 잠재된 의미와 연결된 단어들을 살펴봄으로써 문서를 이루고 있는 잠재 의미가 무엇인지 파악할 수 있다. 이때 잠재된 의미를 토픽이라고 하고, 이러한 분석을 '**토픽 모델링**'이라고 한다.

'**LDA(Latent Dirichlet Allocation)**'가 나오기 전까지 LSA는 이러한 토픽 모델링에도 사용이 됐으나, 현재는 대부분 LDA를 이용한다. LDA를 이용한 토픽 모델링은 뒷 장에서 상세히 다루기로 하고, 여기서는 LSA를 이용한 간단한 토픽 모델링을 살펴본다.

축소된 차원의 수가 너무 크면 토픽을 파악하기가 어려우므로 차원을 10개로 축소하고 각 차원에 대해 단어들의 비중을 살펴보기로 하자. 각 단어에 때해 축소된 차원, 즉 의미의 비중을 TruncatedSVD의 components_ 속성으로 값을 확인할 수 있다. 따라서 아래와 같이 각 토픽별로 비중이 높은 단어 10개를 출력한다.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

RegTok = RegexpTokenizer("[\w']{3,}")
english_stops = set(stopwords.words('english')) 

def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    words = [word for word in tokens if (word not in english_stops) and len(word) > 2]
    features = (list(map(lambda token: PorterStemmer().stem(token), words)))
    return features

tfidf = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [7]:
svd = TruncatedSVD(n_components=10, random_state=1) # 압축할 component의 수 지정
X_train_lsa = svd.fit_transform(X_train_tfidf)
X_test_lsa = svd.transform(X_test_tfidf)

print(f'LSA Converted X shape: {X_train_lsa.shape}')
print(f'Sum of explained variance ratio: {svd.explained_variance_ratio_.sum():.3f}')

terms = tfidf.get_feature_names_out()
def get_topics(model, feature_names, n=10):
    for idx, topic in enumerate(model.components_):
        print(f'Topic {idx+1} {[feature_names[i] for i in topic.argsort()[:-n-1:-1]]}')
get_topics(svd, terms)

LSA Converted X shape: (2034, 10)
Sum of explained variance ratio: 0.045
Topic 1 ['would', 'one', 'god', 'think', 'use', 'peopl', 'know', 'like', 'say', 'space']
Topic 2 ['file', 'imag', 'thank', 'program', 'graphic', 'space', 'format', 'use', 'color', 'ftp']
Topic 3 ['space', 'orbit', 'nasa', 'launch', 'shuttl', 'satellit', 'year', 'moon', 'lunar', 'cost']
Topic 4 ['moral', 'object', 'system', 'valu', 'goal', 'think', 'anim', 'absolut', 'natur', 'defin']
Topic 5 ['ico', 'bobb', 'tek', 'beauchain', 'bronx', 'manhattan', 'sank', 'queen', 'vice', 'blew']
Topic 6 ['god', 'file', 'imag', 'object', 'moral', 'exist', 'space', 'format', 'system', 'color']
Topic 7 ['file', 'islam', 'imag', 'cview', 'use', 'format', 'color', 'muslim', 'religion', 'peopl']
Topic 8 ['post', 'file', 'space', 'islam', 'read', 'cview', 'format', 'articl', 'group', 'moral']
Topic 9 ['christian', 'graphic', 'imag', 'jesu', 'book', 'data', 'group', 'softwar', 'law', 'code']
Topic 10 ['exist', 'atheism', 'atheist', 'gra

토픽이 아주 잘 들어나지는 않지만, Topic 3 같은 경우에는 space, orbit, nasam launce, shuttl 등 주로 'sci.space' 카테고리에 대한 내용들이 잘 나타나 있는 것을 볼 수 있다. 단어별로 비중도 함께 보고 싶다면 feature_names[i]와 함께 topic[i]를 출력한다.

### 단어 간 의미 유사도 분석
LSA를 사용하면 문서 간 유사도와 마찬가지로 단어 간 유사도를 구할 수 있다. 카운트벡터 행렬을 단어의 관점에서 보면, 하나의 단어는 모든 문서에 나타난 빈도로 표현된다고 할 수 있다. 따라서 문서에서 나타나는 빈도 분포에 따라 단어 간의 유사도를 구할 수 있다. LSA에서는 문서가 축소된 의미의 비중으로 표현되므로 이를 이용해 단어 간의 의미적인 유사도를 구할 수 있다. 다만 이때 각 단어의 의미 분포는 주어진 말뭉치에 한정된다는 것을 명심해야 한다.

토픽은 단어들의 비중으로 표현되므로 전처리 과정을 단어 관점에서 다시 할 핋요가 있다. 아래 예시에서는 TFIDF 벡터를 생성할 때 최대 단어수를 1000개로 한정하고, min_df, max_df 값을 각각 5와 0.5로 지정했다. 이후 TruncatedSVD를 이용해 100개의 차원으로 축소했다.

단어들에 대해 잠재된 의미의 비중으로 표현된 값들을 보고 싶다면 np.diag(svd.singularvalues).dot(svd.components_)로 구할 수 있다. 아래 예에서 svd.components_의 shape이 (100, 1000)이므로, 변환된 단어-잠재의미 행렬에 대해 전치 행렬을 구하면 모양은 (100, 1000)이 된다. 즉, 1,000개의 단어에 대해 100개의 내재된 의미를 이용해 유사도를 구할 수 있다.

In [8]:
import numpy as np

tfidf = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train) # train set을 변환

svd = TruncatedSVD(n_components=100, random_state=1) # 압축할 component의 수 지정
X_train_lsa = svd.fit_transform(X_train_tfidf)

print('# components_의 shape:', svd.components_.shape)
print('# singular_values_의 shape:', svd.singular_values_.shape)
t_words = np.diag(svd.singular_values_).dot(svd.components_).T
print('# 변환된 단어-잠재의미 행렬의 shape:', t_words.shape)

# t_words에서 space에 해당하는 벡터를 가져옴
source = t_words[np.where(tfidf.get_feature_names_out() == 'space')[0][0]]
# 변환된 count vector와 기존 값들과의 similarity 계산
sim_result = cosine_similarity([source], t_words)

print('# Top 20 유사도(TFIDF):', sorted(sim_result[0].round(2), reverse=True)[:20])
sim_index = (-sim_result[0]).argsort()[:20]
print('# Top 20 유사 뉴스의 인덱스(TFIDF)', sim_index)
sim_labels = [tfidf.get_feature_names_out()[i] for i in sim_index]
print('# Top 20 유사 뉴스의 카테고리(TFIDF)', sim_labels)

# components_의 shape: (100, 1000)
# singular_values_의 shape: (100,)
# 변환된 단어-잠재의미 행렬의 shape: (1000, 100)
# Top 20 유사도(TFIDF): [1.0, 0.73, 0.72, 0.69, 0.66, 0.58, 0.56, 0.56, 0.54, 0.54, 0.52, 0.52, 0.52, 0.51, 0.51, 0.5, 0.5, 0.49, 0.48, 0.47]
# Top 20 유사 뉴스의 인덱스(TFIDF) [812 314 754 829 594 679 720 650 785 565 101 435 606 545 854 746 669 856
 611 564]
# Top 20 유사 뉴스의 카테고리(TFIDF) ['space', 'exploration', 'sci', 'station', 'office', 'propulsion', 'reports', 'planetary', 'shuttle', 'national', 'astro', 'international', 'operations', 'missions', 'technical', 'satellites', 'probes', 'telescope', 'orbiter', 'nasa']


위 예시에서는 'space'와 유사한 단어들을 검색했고, 당연히 자신에 대해서는 유사도가 1이 나왓으며, 그 외에 'exploration', 'sci', 'station' 등이 순서대로 출력됐다. 이 유사도는 주어진 문서 집합에서 함께 등장한 정도에 따라 계산된 것으로 우리가 생각하는 단어의 의미와는 차이가 있다. 하지만 말뭉치 내에서 단어들이 어떻게 유사한 의미를 갖는지 분석하고 싶을 때 이 방법을 사용하기도 한다.